In [1]:
import sys
sys.path.append('../../..')

In [2]:
import os
import sys
from pathlib import Path
from typing import Tuple
import torch
import numpy as np
from docopt import docopt
from dpu_utils.utils import run_and_debug
from loguru import logger
import pandas as pd
from annoy import AnnoyIndex
from tqdm import tqdm
import shutil
from wandb.apis import InternalApi
import wandb
from codenets.codesearchnet.training_ctx import CodeSearchTrainingContext


/home/mandubian/.cache/pypoetry/virtualenvs/codenets-Xapi4pwr-py3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/mandubian/.cache/pypoetry/virtualenvs/codenets-Xapi4pwr-py3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/mandubian/.cache/pypoetry/virtualenvs/codenets-Xapi4pwr-py3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'

In [7]:
os.environ["WANDB_MODE"] = "dryrun"

In [8]:
restore_dir = "../../../checkpoints/code_search_bert_query_code_siamese_2020_02_12_00_00/code_search_bert_query_code_siamese_2020_02_12_00_00_best_mrr"

In [9]:
logger.debug("Building Training Context")
training_ctx: CodeSearchTrainingContext
logger.info(f"Restoring Training Context from directory{restore_dir}")
training_ctx = CodeSearchTrainingContext.build_context_from_dir(restore_dir)


2020-02-13 16:41:39.389 | DEBUG    | __main__:<module>:1 - Building Training Context
2020-02-13 16:41:39.390 | INFO     | __main__:<module>:3 - Restoring Training Context from directory../../../checkpoints/code_search_bert_query_code_siamese_2020_02_12_00_00/code_search_bert_query_code_siamese_2020_02_12_00_00_best_mrr
2020-02-13 16:41:39.390 | DEBUG    | codenets.recordable:load:226 - RecordableMapping - Loading config
2020-02-13 16:41:39.391 | INFO     | codenets.recordable:load:118 - Loading Config File from ../../../checkpoints/code_search_bert_query_code_siamese_2020_02_12_00_00/code_search_bert_query_code_siamese_2020_02_12_00_00_best_mrr/codenets.codesearchnet.query_code_siamese.training_ctx.QueryCodeSiameseCtx/config/codenets.recordable.HoconConfigRecordable/config.conf
2020-02-13 16:41:39.486 | DEBUG    | codenets.recordable:load:226 - RecordableMapping - Loading model_optimizer
2020-02-13 16:41:39.487 | DEBUG    | codenets.codesearchnet.training_ctx:load:100 - Loading codenet

wandb: Wandb version 0.8.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2020-02-13 16:41:40.676 | INFO     | codenets.codesearchnet.training_ctx:__init__:180 - Re-launching training from epoch: 0 with loss:inf mrr:0.0


In [11]:
queries = pd.read_csv(training_ctx.queries_file)
queries = list(map(lambda q: f"<qy> {q}", queries["query"].values))
queries_tokens, queries_masks = training_ctx.tokenize_query_sentences(
    queries, max_length=training_ctx.conf["dataset.common_params.query_max_num_tokens"]
)
logger.info(f"queries: {queries}")

2020-02-13 16:43:07.091 | INFO     | __main__:<module>:6 - queries: ['<qy> convert int to string', '<qy> priority queue', '<qy> string to date', '<qy> sort string list', '<qy> save list to file', '<qy> postgresql connection', '<qy> confusion matrix', '<qy> set working directory', '<qy> group by count', '<qy> binomial distribution', '<qy> aes encryption', '<qy> linear regression', '<qy> socket recv timeout', '<qy> write csv', '<qy> convert decimal to hex', '<qy> export to excel', '<qy> scatter plot', '<qy> convert json to csv', '<qy> pretty print json', '<qy> replace in file', '<qy> k means clustering', '<qy> connect to sql', '<qy> html encode string', '<qy> finding time elapsed using a timer', '<qy> parse binary file to custom class', '<qy> get current ip address', '<qy> convert int to bool', '<qy> read text file line by line', '<qy> get executable path', '<qy> httpclient post json', '<qy> get inner html', '<qy> convert string to number', '<qy> format date', '<qy> readonly array', '<qy